In [ ]:
import numpy as np
import pandas as pd
import ujson as json
import urllib.request
from concurrent.futures import ThreadPoolExecutor
import requests
import matplotlib.pyplot as plt
import itertools
from collections import Counter
import seaborn as sns
import plotly.express as px
import geopandas as gpd
import folium
from folium.features import Choropleth
import cenpy as c
import networkx as nx
import osmnx as ox
import statsmodels.api as sm
from scipy.stats import chisquare

# DMV EDA with AFDC On Density of EVs and EV Charging Stations Per Zip Code

In [ ]:
# Load DMV data
dmv_df = pd.read_csv('data/DMV_data.csv') 
dmv_df

In [ ]:
# Load AFDC data
afdc_df = pd.read_csv('data/fuel_stations.csv')
afdc_df

In [ ]:
# Extracting the relevant columns in the dataframe for analysis
relevant_columns = ['id', 'access_code', 'fuel_type_code', 'open_date', 'date_last_confirmed', 
                    'state', 'zip', 'city', 'latitude', 'longitude', 'ev_connector_types', 'ev_network'
                    ]
afdc_df = afdc_df[relevant_columns]

# Getting only Electric fuel types
afdc_df_ELEC = afdc_df[afdc_df['fuel_type_code'] == 'ELEC']
afdc_df_ELEC['open_date'] = pd.to_datetime(afdc_df_ELEC['open_date'], errors='coerce')
afdc_df_ELEC

In [ ]:
# Read in csv that contains all the zip codes in the SDGE Territory 
# and creates a list of unique zip codes in SDGE
SDGE_zip_codes_df = pd.read_csv('data/SDGE_zip_codes_2024.csv')
SDGE_zip_codes = SDGE_zip_codes_df['ZipCode'].astype(str).unique()
SDGE_zip_codes = [zip_code.strip() for zip_code in SDGE_zip_codes]

# Create Dataframe of all Vehicles in SDGE Zip codes
dmv_df['ZIP Code'] = dmv_df['ZIP Code'].astype(str)
SDGE_DMV_df = dmv_df[dmv_df['ZIP Code'].isin(SDGE_zip_codes)]

# Creates Dataframe of all EV Chargers in SDGE ZIP Codes
afdc_df_ELEC['zip'] = afdc_df_ELEC['zip'].astype(str).str.strip()
# Remove any leading/trailing spaces
SDGE_chargers_df = afdc_df_ELEC[afdc_df_ELEC['zip'].isin(SDGE_zip_codes)]
SDGE_chargers_df

In [ ]:
# Dataframe of all Vehicles in SDGE Zip codes
SDGE_DMV_df

In [ ]:
# Get the sum of all vehicles registered for each year in the United States and plot it
dmv_vehicles = dmv_df.groupby('Date')['Vehicles'].sum().reset_index(name='Total Number of Vehicles')
fig = px.line(
    dmv_vehicles,
    x='Date',
    y='Total Number of Vehicles',
    title='Total Number of Vehicles Registered by Year in the United States'
)
fig.show()

In [ ]:
# Get the sum of all vehicles registered for each year in the SDGE territory and plot it
SDGE_vehicles = SDGE_DMV_df.groupby('Date')['Vehicles'].sum().reset_index(name='Total Number of Vehicles')
fig = px.line(
    SDGE_vehicles,
    x='Date',
    y='Total Number of Vehicles',
    title='Total Number of Vehicles registered by Year in SDGE Territory'
)
fig.show()

In [ ]:
# Get the sum of all EVs registered for each year in the United States and plot it
ev_df = dmv_df[(dmv_df['Fuel'] == 'Battery Electric') | (dmv_df['Fuel'] == 'Plug-in Hybrid')]
total_EVs = ev_df.groupby('Date')['Vehicles'].sum().reset_index(name='Total Number of Vehicles')

fig = px.line(
    total_EVs,
    x='Date',
    y='Total Number of Vehicles',
    title='Total Number of EVs and plug-in hybrids registered by Year in California'
)
fig.show()

In [ ]:
# Filter DF down to just EVs and Plug-in Hybrids
sdge_ev_df = SDGE_DMV_df[(SDGE_DMV_df['Fuel'] == 'Battery Electric') | (SDGE_DMV_df['Fuel'] == 'Plug-in Hybrid')]

# Get the sum of all EVs registered for each year in the SDGE territory and plot it
SDGE_EVs = sdge_ev_df.groupby('Date')['Vehicles'].sum().reset_index(name='Total Number of Vehicles')
fig = px.line(
    SDGE_EVs,
    x='Date',
    y='Total Number of Vehicles',
    title='Total Number of EVs and plug-in hybrids registered by Year in SDGE Territory'
)
fig.show()

Number of EVs + Plug-in Hybrids (PHEV) expected and predicted from forecast model

2030 Expected: 7.1 million

2030 Predicted: 6.2 million

2035 Expected: 15.2 million

2035 Predicted: 21.1 million

In [ ]:
# Fitting an exponential line to the trend for forecasting
# Step 1: Prepare the data for exponential regression
total_EVs['Log Vehicles'] = np.log(total_EVs['Total Number of Vehicles'])  # Take the log of the vehicle count

# Step 2: Fit a linear regression model to the log-transformed data
X = sm.add_constant(total_EVs['Date'])  # Add a constant to the model (intercept)
y = total_EVs['Log Vehicles']
model = sm.OLS(y, X).fit()

# Step 3: Make predictions and exponentiate to get the original scale
years_forecast = np.arange(total_EVs['Date'].min(), total_EVs['Date'].max() + 12)  # Extend forecast 12 years beyond the last data point
X_forecast = sm.add_constant(years_forecast)
log_y_forecast = model.predict(X_forecast)
y_forecast = np.exp(log_y_forecast)  # Exponentiate to reverse the log transformation

# Step 4: Combine actual data and forecasted data for plotting
forecast_df = pd.DataFrame({'Date': years_forecast, 'Forecasted Vehicles': y_forecast})

# Step 5: Plot the original data and the exponential regression forecast
fig = px.line(
    total_EVs,
    x='Date',
    y='Total Number of Vehicles',
    title='Total Number of EVs and Plug-in Hybrids Registered by Year in California with Exponential Forecast'
)

# Add the forecasted line
fig.add_scatter(x=forecast_df['Date'], y=forecast_df['Forecasted Vehicles'], mode='lines', name='Forecasted Vehicles', line=dict(dash='dash', color='red'))

fig.show()

In [ ]:
# Step 1: Prepare the data for exponential regression
SDGE_EVs['Log Vehicles'] = np.log(SDGE_EVs['Total Number of Vehicles'])  # Take the log of the vehicle count

# Step 2: Fit a linear regression model to the log-transformed data
X = sm.add_constant(SDGE_EVs['Date'])  # Add a constant to the model (intercept)
y = SDGE_EVs['Log Vehicles']
model = sm.OLS(y, X).fit()

# Step 3: Make predictions and exponentiate to get the original scale
years_forecast = np.arange(SDGE_EVs['Date'].min(), SDGE_EVs['Date'].max() + 12)  # Extend forecast to 2035
X_forecast = sm.add_constant(years_forecast)
log_y_forecast = model.predict(X_forecast)
y_forecast = np.exp(log_y_forecast)  # Exponentiate to reverse the log transformation

# Step 4: Combine actual data and forecasted data for plotting
forecast_df = pd.DataFrame({'Date': years_forecast, 'Forecasted Vehicles': y_forecast})

# Step 5: Plot the original data and the exponential regression forecast
fig = px.line(
    SDGE_EVs,
    x='Date',
    y='Total Number of Vehicles',
    title='Total Number of EVs and Plug-in Hybrids Registered by Year in SDGE Territory with Exponential Forecast'
)

# Add the forecasted line
fig.add_scatter(x=forecast_df['Date'], y=forecast_df['Forecasted Vehicles'], mode='lines', name='Forecasted Vehicles', line=dict(dash='dash', color='red'))

fig.show()

## Generate a Poisson Distribution for EVs in SDGE and plot a Monte Carlo histogram

In [ ]:
# Get Total EVs per ZIP Code and Year with a groupby
ev_by_zip_year = sdge_ev_df.groupby(['ZIP Code', 'Date'])['Vehicles'].sum().reset_index()

# Pivot table for poisson fitting
pivot_table = ev_by_zip_year.pivot(index='Date', columns='ZIP Code', values='Vehicles').fillna(0)

# Prepare to store results
poisson_results = []

# Fit Poisson Distribution for each zip code
for zip in pivot_table.columns:
    ev_counts = pivot_table[zip]

    # Only fit if not null
    if ev_counts.sum() > 0:    
        # Fitting Poisson using MLE (Maximum Likelihood Estimation)
        model = sm.GLM(ev_counts, np.ones_like(ev_counts), family=sm.families.Poisson())
        result = model.fit()

        # Get lambda (mean of distribution)
        lambda_estimate = result.mu.mean()
        poisson_results.append({'zip': zip, 'lambda': lambda_estimate})

poisson_df = pd.DataFrame(poisson_results)
poisson_df.to_csv('figures/poisson_sdge.csv', index=False)

poisson_df

In [ ]:
# Monte Carlo simulation
n_samples = 1000
sample_results = {}

# Extract lambda values for each zip code
poisson_params = poisson_df.set_index('zip')['lambda'].to_dict()

# For loop for sampling
for n in range(n_samples):
    results = []
    for zip in poisson_params.keys():
        # Sample from Poisson
        val = poisson.rvs(mu = poisson_params[zip], size=1)
        results.append(val[0])
        sample_results[n] = results
    
# Convert sampling results to Data frame
samples_df = pd.DataFrame.from_dict(sample_results, orient='index', columns=poisson_params.keys())

samples_df.to_csv('figures/monte_carlo.csv', index=False)
samples_df

In [ ]:
# Select the data for ZIP code 91910
zip_code_data = samples_df['91901']

# Plot a histogram
plt.figure(figsize=(10, 6))
plt.hist(zip_code_data, bins=40, edgecolor='k', alpha=0.7)
plt.title('Poisson Distribution of Values for ZIP Code 91901')
plt.xlabel('Number of EVs and PHEVs Registered')
plt.ylabel('Frequency')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# Aggregate data by zip code and year
ev_by_zip_year = ev_df.groupby(['ZIP Code', 'Date'])['Vehicles'].sum().reset_index()
ev_by_zip_year.columns = ['ZIP Code', 'Year', 'Yearly Registrations']
ev_by_zip_year

In [ ]:
# Aggregate EV counts by ZIP code
ev_counts = sdge_ev_df.groupby('ZIP Code')['Vehicles'].sum().reset_index()
ev_counts.columns= ['zip', 'Total EVs']

# Aggregate Charger counts by ZIP code
charger_counts = SDGE_chargers_df.groupby('zip').size().reset_index(name='Charger Count')
charger_counts.sort_values('Charger Count')

In [ ]:
ev_counts.sort_values('Total EVs')

In [ ]:
# Getting the total EV counts per zip code in SDGE in 2024
ev_df_2024 = sdge_ev_df[sdge_ev_df['Date'] == 2024]
ev_counts_2024 = ev_df_2024.groupby('ZIP Code')['Vehicles'].sum().reset_index()
ev_counts_2024.columns = ['zip', 'Total EVs']

In [ ]:
# Merge the total EV counts with charger counts per zip code in SDGE
SDGE_chargers_vehicles = pd.merge(ev_counts_2024, charger_counts, on='zip', how='left').dropna()

# Add new column that get the EV count per Charger
SDGE_chargers_vehicles['EVs per Charger'] = SDGE_chargers_vehicles['Total EVs'] / SDGE_chargers_vehicles['Charger Count'].replace(0, np.nan) 
SDGE_chargers_vehicles

In [ ]:
# Create the scatter plot using plotly.express
fig = px.scatter(
    SDGE_chargers_vehicles, 
    x='Charger Count', 
    y='Total EVs', 
    title='Scatter Plot of Charger Count vs Total EVs per ZIP Code',
    labels={
        'Charger Count': 'Charger Count',
        'Total EVs': 'Total EVs'
    },
    size='Total EVs',  # Size of the points based on Total EVs
    hover_data=['Charger Count', 'Total EVs']  # Display detailed info on hover
)

# Update layout for better readability
fig.update_traces(marker=dict(opacity=0.7, line=dict(width=1, color='DarkSlateGrey')))
fig.update_layout(
    xaxis_title='Charger Count',
    yaxis_title='Total EVs',
    showlegend=False
)

# Fit a linear regression model for the line of best fit
X = SDGE_chargers_vehicles['Charger Count']
X = sm.add_constant(X)  # Add a constant term for the intercept
Y = SDGE_chargers_vehicles['Total EVs']
model = sm.OLS(Y, X).fit()

# Generate the line of best fit
SDGE_chargers_vehicles['Best Fit Line'] = model.predict(X)

# Get the equation of the line
slope = model.params[1]
intercept = model.params[0]
equation = f'Y = {intercept:.2f} + {slope:.2f}X'

# Add the line of best fit to the plot
fig.add_traces(
    px.line(
        SDGE_chargers_vehicles, 
        x='Charger Count', 
        y='Best Fit Line'
    ).data
)

# Update layout for better readability and add equation annotation
fig.update_traces(marker=dict(opacity=0.7, line=dict(width=1, color='DarkSlateGrey')))
fig.update_layout(
    xaxis_title='Charger Count',
    yaxis_title='Total EVs',
    annotations=[
        dict(
            x=0.5,
            y=0.95,
            xref='paper',
            yref='paper',
            text=equation,
            showarrow=False,
            font=dict(size=14)
        )
    ],
    showlegend=False
)

# Show the interactive plot
fig.show()


# Folium Map Showing the EV Vehicle Density Relative to the number of EV Chargers per Zip Code 

In [ ]:
# Load the shapefile containing ZIP code boundaries for all of US
zip_shapefile = gpd.read_file('data/tl_2024_us_zcta520/tl_2024_us_zcta520.shp')
# Ensure that the ZIP code column in both dataframes are of the same type (string)
SDGE_chargers_vehicles['zip'] = SDGE_chargers_vehicles['zip'].astype(str)
zip_shapefile['ZCTA5CE20'] = zip_shapefile['ZCTA5CE20'].astype(str)

# Merge the charger data with the ZIP code shapefile
map_zip_chargers = zip_shapefile.merge(SDGE_chargers_vehicles, left_on='ZCTA5CE20', right_on='zip', how='inner')

# Fill NaN values with 0 for ZIP codes that have no chargers
map_zip_chargers['Charger Count'] = map_zip_chargers['Charger Count'].fillna(0)
map_zip_chargers = map_zip_chargers.sort_values('Charger Count', ascending=False)
map_zip_chargers

In [ ]:
# Calculate the density of Total EVs per Charger and fill NaNs with 0
map_zip_chargers['EVs per Charger Density'] = map_zip_chargers['EVs per Charger'].fillna(0)

# Create a base map centered around the SDGE region
m = folium.Map(location=[32.7157, -117.1611], zoom_start=10)

# Add a choropleth layer for EVs per Charger Density
Choropleth(
    geo_data=map_zip_chargers,
    data=map_zip_chargers,
    columns=['zip', 'EVs per Charger Density'],
    key_on='feature.properties.ZCTA5CE20',
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='EVs per Charger Density per ZIP Code'
).add_to(m)

# Add tooltips for interactivity
folium.GeoJson(
    map_zip_chargers,
    style_function=lambda feature: {
        'fillColor': 'transparent',
        'color': 'black',
        'weight': 0.5
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['zip', 'Charger Count', 'Total EVs', 'EVs per Charger Density'],
        aliases=['ZIP Code:', 'Charger Count:', 'Total EVs:', 'EVs per Charger Density:'],
        localize=True
    )
).add_to(m)

# Save and display the map
m.save('figures/choropleth_map_evs_per_charger.html')
m